In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
# 1. Create an Image Data Generator
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,           # Rescale pixel values to [0, 1]
    shear_range=0.2,          # Randomly shear images
    zoom_range=0.2,           # Randomly zoom into images
    horizontal_flip=True,     # Randomly flip images horizontally
    validation_split=0.2      # Use 20% of images for validation
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load training and validation data
train_generator = train_datagen.flow_from_directory(
    '../images_ternary_bandpass',             # Directory with training data
    target_size=(775, 616),   
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',      # For categorical classification
    subset='training'         # Use the training subset
)

validation_generator = train_datagen.flow_from_directory(
    '../images_ternary_bandpass',             # Directory with validation data
    target_size=(775, 616),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'       # Use the validation subset
)

# 2. Build the CNN Model
model = tf.keras.models.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(775, 616, 1)),
    MaxPooling2D(pool_size=(3, 3)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(3, activation='softmax')  # For categorical classification
])

# 3. Compile the Model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 4. Define Callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '../models/best_model.keras',          # Save the best model
    monitor='val_accuracy',   # Monitor validation accuracy
    verbose=1,
    save_best_only=True,      # Save only the best model
    mode='max'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',   # Monitor validation accuracy
    patience=5,               # Stop if no improvement after 5 epochs
    verbose=1,
    
    mode='max'
)

callbacks = [checkpoint, early_stopping]

# 5. Train the Model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15,                 # Maximum number of epochs
    callbacks=callbacks        # Use the callbacks defined above
)

Found 1223 images belonging to 3 classes.
Found 303 images belonging to 3 classes.


C:\Users\joshu\miniconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15


C:\Users\joshu\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4058 - loss: 1.2912
Epoch 1: val_accuracy improved from -inf to 0.42014, saving model to ../models/best_model.keras
38/38 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.4061 - loss: 1.2868 - val_accuracy: 0.4201 - val_loss: 1.0138
Epoch 2/15
 1/38 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 0.4375 - loss: 1.0023
Epoch 2: val_accuracy did not improve from 0.42014


C:\Users\joshu\miniconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4375 - loss: 1.0023 - val_accuracy: 0.2667 - val_loss: 1.1638
Epoch 3/15
 2/38 ━━━━━━━━━━━━━━━━━━━━ 1:01 2s/step - accuracy: 0.4609 - loss: 0.9891


KeyboardInterrupt

